### Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from ingest.catalog import DatasetCatalog
from ingest.loader import RawDatasetLoader
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.excel import ExcelAdapter
from ingest.fetch.metadata import SocrataMetadata
from utils.clean import normalize_text
from config import DATA_PATH

In [3]:
## servicios
catalog = DatasetCatalog()
loader = RawDatasetLoader(
    csv_adapter=CsvAdapter(),
    sct_adapter=SocrataAdapter(),
    excel_adapter=ExcelAdapter(),
)

### Info

In [4]:
ds = catalog.get("pib_municipios")
meta = loader.metadata(ds)
if isinstance(meta, SocrataMetadata):
    print("Nombre:", meta.name)
    print("Descripción:", meta.description)
    print("Columnas:")
    columns = meta.columns
    for col in columns:
        print(f"  - {col.name} ({col.fieldName}): {col.dataTypeName}")
else:
    print("Este origen no soporta metadata estructurada.")

Nombre: PIB Departamental con proyección
Descripción: Presenta la proyección del Producto Interno Bruto (PIB) constante, de los Departamentos de Colombia,  expresado en precios de 2015.
Columnas:
  - Año (a_o): number
  - Actividad (actividad): text
  - Sector (sector): text
  - Tipo de precios (tipo_de_precios): text
  - Código Departamento (DIVIPOLA) (c_digo_departamento_divipola): number
  - Departamento (departamento): text
  - Valor (miles de millones de pesos) (valor_miles_de_millones_de): number


In [5]:
records = list(loader.load(ds))
df = pd.DataFrame(records)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16302 entries, 0 to 16301
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   a_o                           16302 non-null  object
 1   actividad                     16302 non-null  object
 2   sector                        16302 non-null  object
 3   tipo_de_precios               16302 non-null  object
 4   c_digo_departamento_divipola  16302 non-null  object
 5   departamento                  16302 non-null  object
 6   valor_miles_de_millones_de    16302 non-null  object
dtypes: object(7)
memory usage: 891.6+ KB


### Cleaning

In [6]:
ds = catalog.get("ciiu")
df_ciiu = pd.DataFrame(list(loader.load(ds)))

ds = catalog.get("divipola")
df_div = pd.DataFrame(list(loader.load(ds)))

print("CIUU:", df_ciiu.shape)
print("DIVIPOLA:", df_div.shape)

CIUU: (495, 9)
DIVIPOLA: (1122, 7)


In [ ]:
MACRO_MAP_PIB = {
    "CONSTRUCCION": "CONSTRUCCIÓN",
    "INFORMACION Y COMUNICACIONES": "SERVICIOS",
    "AGRICULTURA GANADERIA CAZA SILVICULTURA Y PESCA": "AGROPECUARIO",
    "EXPLOTACION DE MINAS Y CANTERAS": "MINERO",
    "INDUSTRIAS MANUFACTURERAS": "MANUFACTURA",
    "SUMINISTRO DE ELECTRICIDAD GAS VAPOR Y AIRE ACONDICIONADO; DISTRIBUCION DE AGUA; EVACUACION Y TRATAMIENTO DE AGUAS RESIDUALES; GESTION DE DESECHOS Y ACTIVIDADES DE SANEAMIENTO AMBIENTAL": "SERVICIOS",
    "COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS; TRANSPORTE Y ALMACENAMIENTO; ALOJAMIENTO Y SERVICIOS DE COMIDA": "SERVICIOS",
    "ACTIVIDADES FINANCIERAS Y DE SEGUROS": "SERVICIOS",
    "ACTIVIDADES INMOBILIARIAS": "SERVICIOS",
    "ACTIVIDADES PROFESIONALES CIENTIFICAS Y TECNICAS; ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO": "SERVICIOS",
    "ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA; EDUCACION; ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE SERVICIOS SOCIALES": "SERVICIOS",
    "ACTIVIDADES ARTISTICAS DE ENTRETENIMIENTO Y RECREACION Y OTRAS ACTIVIDADES DE SERVICIOS; ACTIVIDADES DE LOS HOGARES INDIVIDUALES EN CALIDAD DE EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES INDIVIDUALES COMO PRODUCTORES DE BIENES Y SERVICIOS PARA USO PROPIO": "SERVICIOS",
    "IMPUESTOS": "SERVICIOS"
}

In [ ]:
def infer_macro_pib(actividad):
    if actividad is None:
        return None
    key = normalize_text(actividad)
    return MACRO_MAP_PIB.get(key, None)

df_pib = df.rename(columns={
    "a_o": "year",
    "c_digo_departamento_divipola": "dept_code",
    "valor_miles_de_millones_de": "value"
})

df_pib["dept_code"] = (
    df_pib["dept_code"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.zfill(2)
)

def clean_number(x):
    if pd.isna(x):
        return None
    x = str(x)
    x = x.replace("$", "").replace(" ", "")
    x = x.replace(",", ".")
    return pd.to_numeric(x, errors="coerce")

In [ ]:
df_pib["year"] = df_pib["year"].apply(clean_number)
df_pib["value"] = df_pib["value"].apply(clean_number)

df_div_dept = (
    df_div[["cod_dpto"]]
    .drop_duplicates()
    .rename(columns={"cod_dpto": "dept_code"})
)

df_div_dept["dept_code"] = df_div_dept["dept_code"].astype(str).str.zfill(2)

df_pib["actividad_norm"] = df_pib["actividad"].apply(normalize_text)
df_pib["macrosector"] = df_pib["actividad_norm"].apply(infer_macro_pib)

df_sector_year = df_pib[["year", "dept_code", "macrosector", "value"]].dropna()

df_sector_year = df_sector_year[
    (df_sector_year["year"] > 1900) &
    (df_sector_year["value"] > 0)
].reset_index(drop=True)

df_sector_year.to_csv(f"{DATA_PATH}/processed/pib_sector_year.csv", index=False)